In [1]:
import sqlite3
from getpass import getpass
import random

In [2]:
conn = sqlite3.connect(":memory:")  # Use ":memory:" for temporary database or "exam_system.db" for persistent storage
cursor = conn.cursor()

In [3]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS students (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL,
    password TEXT NOT NULL
)
""")
cursor.execute("""
CREATE TABLE IF NOT EXISTS subjects (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    description TEXT
)
""")
cursor.execute("""
CREATE TABLE IF NOT EXISTS exams (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    subject_id INTEGER NOT NULL,
    date TEXT NOT NULL,
    time_limit INTEGER NOT NULL,
    FOREIGN KEY(subject_id) REFERENCES subjects(id)
)
""")
cursor.execute("""
CREATE TABLE IF NOT EXISTS questions (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    exam_id INTEGER NOT NULL,
    question_text TEXT NOT NULL,
    option1 TEXT NOT NULL,
    option2 TEXT NOT NULL,
    option3 TEXT NOT NULL,
    option4 TEXT NOT NULL,
    correct_option INTEGER NOT NULL,
    FOREIGN KEY(exam_id) REFERENCES exams(id)
)
""")
cursor.execute("""
CREATE TABLE IF NOT EXISTS results (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    student_id INTEGER NOT NULL,
    exam_id INTEGER NOT NULL,
    score INTEGER NOT NULL,
    FOREIGN KEY(student_id) REFERENCES students(id),
    FOREIGN KEY(exam_id) REFERENCES exams(id)
)
""")
conn.commit()

In [4]:
def add_student():
    name = input("Enter student name: ")
    email = input("Enter student email: ")
    password = getpass("Enter password: ")
    try:
        cursor.execute("INSERT INTO students (name, email, password) VALUES (?, ?, ?)", (name, email, password))
        conn.commit()
        print("Student added successfully!")
    except sqlite3.IntegrityError:
        print("Email already exists!")

In [5]:
def add_subject():
    name = input("Enter subject name: ")
    description = input("Enter subject description: ")
    cursor.execute("INSERT INTO subjects (name, description) VALUES (?, ?)", (name, description))
    conn.commit()
    print("Subject added successfully!")

In [6]:
def add_exam():
    subject_id = int(input("Enter subject ID: "))
    date = input("Enter exam date (YYYY-MM-DD): ")
    time_limit = int(input("Enter time limit (minutes): "))
    cursor.execute("INSERT INTO exams (subject_id, date, time_limit) VALUES (?, ?, ?)", (subject_id, date, time_limit))
    conn.commit()
    print("Exam added successfully!")


In [7]:
def add_question():
    exam_id = int(input("Enter exam ID: "))
    question_text = input("Enter question: ")
    option1 = input("Enter option 1: ")
    option2 = input("Enter option 2: ")
    option3 = input("Enter option 3: ")
    option4 = input("Enter option 4: ")
    correct_option = int(input("Enter the correct option number (1-4): "))
    cursor.execute("INSERT INTO questions (exam_id, question_text, option1, option2, option3, option4, correct_option) VALUES (?, ?, ?, ?, ?, ?, ?)",
                   (exam_id, question_text, option1, option2, option3, option4, correct_option))
    conn.commit()
    print("Question added successfully!")

In [8]:
def student_login():
    email = input("Enter your email: ")
    password = getpass("Enter your password: ")
    cursor.execute("SELECT id FROM students WHERE email = ? AND password = ?", (email, password))
    student = cursor.fetchone()
    if student:
        print("Login successful!")
        return student[0]
    else:
        print("Invalid credentials.")
        return None

def take_exam(student_id):
    exam_id = int(input("Enter the exam ID you want to take: "))
    cursor.execute("SELECT * FROM questions WHERE exam_id = ?", (exam_id,))
    questions = cursor.fetchall()
    if not questions:
        print("No questions found for this exam.")
        return
    score = 0
    for q in questions:
        print(f"\n{q[2]}")
        print(f"1) {q[3]}")
        print(f"2) {q[4]}")
        print(f"3) {q[5]}")
        print(f"4) {q[6]}")
        answer = int(input("Enter your answer (1-4): "))
        if answer == q[7]:
            score += 1
    cursor.execute("INSERT INTO results (student_id, exam_id, score) VALUES (?, ?, ?)", (student_id, exam_id, score))
    conn.commit()
    print(f"Exam completed! Your score: {score}")

In [9]:
def main():
    while True:
        print("\nMain Menu:")
        print("1. Admin: Add Student")
        print("2. Admin: Add Subject")
        print("3. Admin: Add Exam")
        print("4. Admin: Add Question")
        print("5. Student: Login and Take Exam")
        print("6. Exit")
        choice = input("Enter your choice: ")
        if choice == '1':
            add_student()
        elif choice == '2':
            add_subject()
        elif choice == '3':
            add_exam()
        elif choice == '4':
            add_question()
        elif choice == '5':
            student_id = student_login()
            if student_id:
                take_exam(student_id)
        elif choice == '6':
            print("Exiting...")
            break
        else:
            print("Invalid choice!")

if __name__ == "__main__":
    main()


Main Menu:
1. Admin: Add Student
2. Admin: Add Subject
3. Admin: Add Exam
4. Admin: Add Question
5. Student: Login and Take Exam
6. Exit
Enter your choice: 1
Enter student name: Vansh
Enter student email: vansh@gmail.com
Enter password: ··········
Student added successfully!

Main Menu:
1. Admin: Add Student
2. Admin: Add Subject
3. Admin: Add Exam
4. Admin: Add Question
5. Student: Login and Take Exam
6. Exit
Enter your choice: 3
Enter subject ID: 5656
Enter exam date (YYYY-MM-DD): 2025-08-28
Enter time limit (minutes): 180
Exam added successfully!

Main Menu:
1. Admin: Add Student
2. Admin: Add Subject
3. Admin: Add Exam
4. Admin: Add Question
5. Student: Login and Take Exam
6. Exit
Enter your choice: 5
Enter your email: vansh@gmail.com
Enter your password: ··········
Login successful!
Enter the exam ID you want to take: 5656
No questions found for this exam.

Main Menu:
1. Admin: Add Student
2. Admin: Add Subject
3. Admin: Add Exam
4. Admin: Add Question
5. Student: Login and Take 